# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import sqlite3
import pandas as pd


In [2]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [4]:
sql_str = """SELECT c.customerNumber
             , c.contactLastName
             , c.contactFirstName
             FROM customers c
             WHERE c.customerNumber IN
                 (SELECT customerNumber
                  FROM orders
                  WHERE orderDate = '2003-01-31')
             ;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [10]:
sql_str = """SELECT p.productName
             , COUNT(o.orderNumber) AS total_orders
             , SUM(o.quantityOrdered) AS items_sold
             FROM products p
             JOIN orderdetails o USING (productCode)
             GROUP BY productName
             ORDER BY items_sold
             ;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,productName,total_orders,items_sold
0,1957 Ford Thunderbird,24,767
1,1970 Chevy Chevelle SS 454,25,803
2,1936 Mercedes Benz 500k Roadster,25,824
3,1911 Ford Town Car,25,832
4,1999 Indy 500 Monte Carlo SS,25,855
...,...,...,...
104,1930 Buick Marquette Phaeton,28,1074
105,1941 Chevrolet Special Deluxe Cabriolet,28,1076
106,American Airlines: MD-11S,28,1085
107,1937 Lincoln Berline,28,1111


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [16]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
sql_str = """SELECT p.productName
             , COUNT(DISTINCT c.customerName) as customers_ordered
             FROM products p
             JOIN orderdetails od USING (productCode)
             JOIN orders o USING (orderNumber)
             JOIN customers c USING (customerNumber)
             GROUP BY p.productName
             ORDER BY customers_ordered DESC
             ;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,productName,customers_ordered
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [56]:
# Your code here

sql_str = """SELECT DISTINCT e.employeeNumber
             , e.lastName
             , e.firstName
             , off.city
             , off.officeCode
             FROM employees e
             JOIN offices off USING (officeCode)
             JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
             JOIN orders o USING (customerNumber)
             JOIN orderdetails od USING (orderNumber)
             WHERE od.productCode IN
                 (SELECT od.productCode
                  FROM orders o
                  JOIN orderdetails od USING (orderNumber)
                  GROUP BY od.productCode
                  HAVING COUNT(DISTINCT o.customerNumber) < 20)
             ;"""



df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,employeeNumber,lastName,firstName,city,officeCode
0,1370,Hernandez,Gerard,Paris,4
1,1501,Bott,Larry,London,7
2,1337,Bondur,Loui,Paris,4
3,1166,Thompson,Leslie,San Francisco,1
4,1286,Tseng,Foon Yue,NYC,3
5,1612,Marsh,Peter,Sydney,6
6,1611,Fixter,Andy,Sydney,6
7,1401,Castillo,Pamela,Paris,4
8,1621,Nishi,Mami,Tokyo,5
9,1323,Vanauf,George,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [71]:
sql_str = """SELECT e.employeeNumber
             , e.lastName
             , e.firstName
             , COUNT(c.customerName) AS customerCreditAvgOver15k
             FROM employees e
             JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
             GROUP BY e.employeeNumber
             HAVING AVG(c.creditLimit) > 15000
             ;"""

df = pd.DataFrame(cur.execute(sql_str).fetchall())
df.columns = [x[0] for x in cur.description]

df

,employeeNumber,lastName,firstName,customerCreditAvgOver15k
0,1165,Jennings,Leslie,6
1,1166,Thompson,Leslie,6
2,1188,Firrelli,Julie,6
3,1216,Patterson,Steve,6
4,1286,Tseng,Foon Yue,7
5,1323,Vanauf,George,8
6,1337,Bondur,Loui,6
7,1370,Hernandez,Gerard,7
8,1401,Castillo,Pamela,10
9,1501,Bott,Larry,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!